In [2]:
import mysql.connector
from mysql.connector import Error
import pandas as pd
import os
import json
import requests
from sqlalchemy import create_engine

# Global properties
startdate = '2024-06-01'
enddate = '2024-06-30'
period = '202406'

# Folder containing SQL query files
sql_folder = 'api_scripts'

# MySQL database connection details
db_user = 'root'
db_password = 'test' # Set mysql password
db_host = 'localhost'
db_port = '3306'  # Default MySQL port is 3306
db_name = 'ldwh'
table_name = 'dataset_values'

# Create a database connection engine
engine = create_engine(f'mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

# Replace with your DHIS2 instance URL and credentials
dhis2_url = 'LDMS URL'
username = 'LDMS USER'
password = 'LDMS PASSWORD'

# Function to execute a stored procedure
def execute_stored_procedure(cursor, proc_name):
    try:
        cursor.callproc(proc_name)
        print(f"Stored procedure {proc_name} executed successfully.")
    except Error as e:
        print(f"Error executing stored procedure {proc_name}: {e}")

# Function to execute SQL queries from files
def execute_sql_file(cursor, file_path, params):
    try:
        with open(file_path, 'r') as file:
            query = file.read()
        
        # Replace placeholders with actual values
        query = query.format(startDate=params['startdate'], endDate=params['enddate'], period=params['period'])
        
        for result in cursor.execute(query, multi=True):
            if result.with_rows:
                print(f"Rows produced by query: {result.fetchall()}")
            else:
                print(f"Number of rows affected: {result.rowcount}")

        print(f"SQL file {file_path} executed successfully.")
    except Error as e:
        print(f"Error executing SQL file {file_path}: {e}")
    except FileNotFoundError:
        print(f"File {file_path} not found.")

# Function to read database names from a CSV file
def read_databases_from_csv(file_path):
    try:
        df = pd.read_csv(file_path)
        return df['database'].tolist()
    except Exception as e:
        print(f"Error reading database names from CSV: {e}")
        return []
    
# Function to post data to DHIS2
def post_data_to_dhis2():
    try:
        # Read data from MySQL table into a DataFrame
        query = f'SELECT * FROM {table_name}'
        df = pd.read_sql(query, engine)

        # Remove any rows with null values
        df = df.dropna()

        # Convert the DataFrame to the required JSON format
        data_values = [
            {
                "dataElement": row['data_element'],
                "categoryOptionCombo": row['category_option'],
                "orgUnit": row['organization_unit'],
                "period": row['period'],
                "value": row['value']
            }
            for _, row in df.iterrows()
        ]

        data_value_set = {
            "dataValues": data_values
        }

        json_data = json.dumps(data_value_set, indent=2)

        # Headers for the request
        headers = {
            'Content-Type': 'application/json'
        }

        # Make the POST request to the DHIS2 API
        response = requests.post(dhis2_url, headers=headers, data=json_data, auth=(username, password))

        # Print the response from the server
        print(response.status_code)
        print(response.json())
    except Exception as e:
        print(f"Error posting data to DHIS2: {e}")

# Main function to run the pipeline
def run_pipeline():
    databases = read_databases_from_csv('databases.csv')
    if not databases:
        print("No databases to process.")
        return

    sql_files = [os.path.join(sql_folder, f) for f in os.listdir(sql_folder) if f.endswith('.sql')]

    for db in databases:
        try:
            # Connect to the database
            connection = mysql.connector.connect(
                host='localhost',
                user='root',
                password='test',
                database=db
            )
            if connection.is_connected():
                cursor = connection.cursor()
                print(f"Connected to database {db}")

                # Execute stored procedures
                execute_stored_procedure(cursor, 'create_etl_tables')
                execute_stored_procedure(cursor, 'sp_first_time_setup')

                # Execute SQL queries from files
                params = {'startdate': startdate, 'enddate': enddate, 'period': period}
                for sql_file in sql_files:
                    execute_sql_file(cursor, sql_file, params)

                # Commit the transactions
                connection.commit()
                print(f"Pipeline completed for database {db}")

        except Error as e:
            print(f"Error connecting to database {db}: {e}")

        finally:
            if connection.is_connected():
                cursor.close()
                connection.close()
                print(f"Connection to database {db} closed")
    
    # post data to dhis2
    post_data_to_dhis2()

if __name__ == '__main__':
    run_pipeline()
    


Connected to database amani_medical
Stored procedure create_etl_tables executed successfully.
Stored procedure sp_first_time_setup executed successfully.
Number of rows affected: 0
Number of rows affected: 0
Number of rows affected: 0
Number of rows affected: 7
SQL file api_scripts\Current on ART.sql executed successfully.
Number of rows affected: 0
Number of rows affected: 0
Number of rows affected: 0
Number of rows affected: 0
SQL file api_scripts\HTS-OPD Tested.sql executed successfully.
Pipeline completed for database amani_medical
Connection to database amani_medical closed
Connected to database openmrs
Stored procedure create_etl_tables executed successfully.
Stored procedure sp_first_time_setup executed successfully.
Number of rows affected: 0
Number of rows affected: 0
Number of rows affected: 0
Number of rows affected: 30
SQL file api_scripts\Current on ART.sql executed successfully.
Number of rows affected: 0
Number of rows affected: 0
Number of rows affected: 0
Number of row